In [169]:
import os

In [170]:
%pwd

'/Users/chaitanyachangala/Desktop/Chicken-Disease-classifier'

In [171]:
os.chdir('/Users/chaitanyachangala/Desktop/Chicken-Disease-classifier/')
%pwd

'/Users/chaitanyachangala/Desktop/Chicken-Disease-classifier'

In [172]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [173]:
from src.cnn_classifier.constants import *
from src.cnn_classifier.utils.common import read_yaml
from src.cnn_classifier.utils.common import create_directories

print()

In [174]:
class configurationManager():
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_url = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )

In [175]:
import os
import urllib.request as request
import zipfile
from src.cnn_classifier import logger
from src.cnn_classifier.utils.common import get_size

In [176]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_url, 
                filename = self.config.local_data_file)
            logger.info(f"File downloaded at {filename} with info {header}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists of size {get_size(Path(self.config.local_data_file))}")

    def unzip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path) 


In [177]:
try:
    config = configurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    logger.exception(e)
    raise e

[2025-03-31 21:50:43,519]: INFO:common: yaml file: config/config.yaml loaded successfully
[2025-03-31 21:50:43,522]: INFO:common: yaml file: params.yaml loaded successfully
[2025-03-31 21:50:43,522]: INFO:common: created directory at: artifacts
[2025-03-31 21:50:43,523]: INFO:common: created directory at: artifacts/data_ingestion
[2025-03-31 21:50:44,731]: INFO:3942586820: File downloaded at artifacts/data_ingestion/data.zip with info Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BF60:3C4A77:166413E:1C4D12C:67EB42E7
Accept-Ranges: bytes
Date: Tue, 01 Apr 2025 01:50:43 GMT
Via: 1.1 varnish
X-Served-By: cache-bos4667-BOS
X-Cache: HIT
X-